In [ ]:
---
title: "The JavaScript Event Loop"
author: "Vahram Poghosyan"
date: "2025-04-15"
categories: ["JavaScript", "Concurrency", "Event Loop"]
format:
  html:
    code-fold: false
    code-line-numbers: false
    code-tools:
      source: https://github.com/v-poghosyan/manim-projects
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

### Single - Threadedness

Post 1/2 from transfer on Nov 3, 2025. 

JavaScript is a *single-threaded* language, which basically means it can execute one statement at a time. 

Consider the following code: 

```javascript
console.log("Hello world!");
alert("!!!");
console.log("Goodbye world!");
```

The second `console.log()` is not executed until the `alert` is dismissed by the user. In other words, JavaScript waits until execution of `line 2` finishes before moving on to `line 3`. 

This can pose severe limitations when doing things that take time (e.g. API requests, waiting on user input, etc.) We don't want JavaScript to just freeze up and wait for these things.

Luckily, there is a solution to this problem implemented in the browsers themselves.

### Asynchronous Code in JavaScript - Web APIs and The Event Loop

Consider how JavaScript handles timers.

```JavaScript
console.log("I'm first");
setTimeout(() => {console.log("I'm third");}, 3000) // Prints after a 3 second delay
console.log("I'm second");
```

The corresponding output of the above code is:

```
I'm first
I'm second
I'm third
```

But if JavaScript is a single-threaded language, how does it execute `line 3` without first finishing the execution of `line 2`? 

#### Enter the browser...

Browsers are usually built using *multi-threaded* languages (typically C++) so they're capable of keeping track of multiple things at once. They come with a set of APIs, called *Web APIs*, that are able to handle a whole host of tasks in the background for JavaScript. These could be things like setting timers, making requests, etc.

In the above example, when `setTimeout()` is called, its functional argument `() => {console.log("I'm third")}` is passed off to the browser's *Timer API* which keeps track of the time and reminds JavaScript to call that function when appropriate. It does so by first pushing the function to the browsers' *callback queue*, then popping it and pushing it to JavaScript's own *call stack*. This is carried out by a background process called *the event loop* which constantly monitors the *callback queue* and the *call stack*.

> **Note:** the *callback queue* is also what's used to execute simple callback functions. The callback is first placed in the *callback queue* by the browser. Then the *event loop* determines if the parent function (the one calling the callback) has been popped from the *call stack*. If it has, then the callback function is pushed to the *call stack*.



### The Callback Hell

Before introducing *Promises*, it's important to understand how callbacks can get messy. 

Suppose we have a function that increments a variable by `num`. In order to repeatedly increment the variable, we'd have to either call this function in a loop (which would be the reasonable thing to do), or pass it to itself as a callback multiple times. For the moment we are not concerned with being reasonable — our goal is to produce messy code. So, we define a function that accepts a functional parameter `callback`:

```javascript
let result = 0; // Global variable that stores the result

function increment(num, callback = null) {
  
	result += num;
	if (callback) {callback();}
  
}
```

Now, if we want to increment `result` to `15`, we could do:

```javascript
increment(
  5,
  increment(
    5,
    increment(5)
  )
);

console.log(`Result is: ${result}`)
```

```
Result is: 15
```

We can see the nestedness quickly getting out of hand. This is refered to, quite dramatically, as *the callback hell* or *the pyramid of doom*. 

Of course, nobody in their right mind would ever increment a variable this way so it's a rather artificial example. However, chaining asynchronous callbacks in general can naturally result in this pattern — especially when multiple callbacks are passed instead of one. For example, when repeatedly making HTTP requests, we have two asynchronous callbacks as parameters to each request: one handling the case of a successful request and another one handling the case of a request failure.

Let's add a failure state to the above `increment()` function. Suppose we want to check for whether or not `result` has surpassed the value of  `20` and, if so, disallow further incrementation. Here's one way we could extend the function:

```javascript
let result = 0; // Global variable that stores the result

function increment(num, callback = null) {
  
  if (result + num <= 20) {
    result += num;
  } else {
    console.log("I will not increment the value anymore!"); // Backed-in behavior
  }
  
	if (callback) {callback();}
  
}
```

This works, but what if we want custom (as opposed to baked-in) behavior for when the `result` surpasses the value of `20`? After all, that's certainly the case when making HTTP requests with a built-in JavaScript function that supports user-defined behavior in case of a request failure. 

The solution is to pass two callback functions instead of one, `onSuccess` and `onFailure`:

 ```javascript
 let result = 0; // Global variable that stores the result
 
 function increment(num, onSuccess = null, onFailure = null) {
   
   if (result + num <= 20) {
     result += num;
     if (onSuccess) {onSuccess();}
   } 
   else {
     if (onFailure) {onFailure()};
   }
   
 }
 ```

Now in the outter lexical environment, we'd call this function as follows: 

```javascript
increment(
  5,
  // On success... (1)
  increment(
    5,
    // On success... (2)
    increment(5)
    // On failure... (2)
    () => {console.log("I will not increment the value anymore!");}
  ),
  // On failure... (1)
  () => {console.log("I will not increment the value anymore!");}
);

console.log(`Result is: ${result}`)
```

```
Result is: 15
```

We can now see the added complexity of passing two callback functions instead of one. If we weren't convinced of the hellishness of *the callback hell* before, hopefully this example did the trick...



### Promises - Avoiding the Callback Hell

*Promises* are JavaScript objects that eventually evaluate to either a status of `"resolved"` or a status of `"rejected"`, representing the successful completion of a process or its failure.

First, let's see how to work with  promises — how to create one, how to *resolve* or *reject* it, and how to *consume* it.

#### Creating a Promise

Suppose your roomate promises to wash the dishes after a house party. Programmatically, we would represent his/her promise like so:

```javascript
const willWashTheDishes = new Promise((resolve, reject) => {});
```

The `Promise` constructor accepts a functional parameter called the *executor*, `(resolve, reject) => {}`, which, in turn, accepts two functional parameters `resolve()` and `reject()`. The executor is what determines the behavior of the `Promise` object. 

If we invoke neither `resolve()` nor `reject()` in the body of the function, then we get the following *Promise* object:

```javascript
console.log(willWashTheDishes)
```

```
> Promise {<pending>}
	> __proto__: Promise
	[[PromiseStatus]]: "pending"
	[[PromiseValue]]: undefined
```

As we can see the status of the promise is *"pending"*. That is, it's still subject to either resolution or rejection. 

#### Resolving or Rejecting a Promise

As one would expect, if we invoke `resolve()` in the body of the function, the status will be `"resolved"` (indicating that the roomate *has* washed the dishes). Similarly, if we invoke  `reject()` the status will be `"rejected"` (indicating that the roomate *has not* washed the dishes).

Let's say the roomate has a 50/50 chance of keeping his promise. The outcome of his decision could be modelled programmatically like so: 

```javascript
const willWashTheDishes = new Promise((resolve, reject) => {
  
  const rand = Math.random();
  
  if (rand < 0.5) {resolve();}
  else {reject();}
  
});
```

Of course, this is a contrived example where the promise is immediately either resolved or rejected. There's no asynchronicity, so we could simply have written conditional logic instead of creating a `Promise` object. In practice, however, a promise may be *"pending"* for a while before it's finally resolved or rejected. This is the case, for example, when making API requests to a server. 

#### Consuming a Promise

Consuming a promise means running code conditionally based on the status of the promise. Each *Promise* object has a `then()` *consumer* (an instance method of the `Promise` object) which accepts a functional parameter that runs automatically when the promise has been resolved. Similarly, each *Promise* object has a `catch()` consumer which also accepts a functional parameter that runs automatically when it has been rejected.

Let's see these in action: 

```javascript
willWashTheDishes.then(() => { console.log("The dishes are clean!"); });
willWashTheDishes.catch(() => { console.log("The dishes are dirty!"); });
```

We can also chain `catch()` to`then()`, like so:

```javascript
willWashTheDishes
	.then(() => { console.log("The dishes are clean!"); })
	.catch(() => { console.log("The dishes are dirty!"); });
```

> **Note:** The functional parameters of these consumers obviously fill the role of the callbacks `onSuccess` and `onFailure` in *the callback hell* scenario.

#### Resolving or Rejecting a Promise with a Value

The executor's parameters `resolve()` and `reject()` also accept optional parameters. 

For example, we could reject a promise with a status code (a reason for the failure). By convention, we encapsulate the status code into an object of the following form:

```javascript
reject({status: "The dishes were all broken!"})
```

Or, we can resolve a promise with a status code and the corresponding data, say the fraction of the dishes that were washed:

```javascript
resolve({status: "The dishes were washed!", data: 1.0})
```

The consumers `then()` and `catch()` will now have access to these returned values within their callback funtions:

```javascript
willWashTheDishes.then((res) => { console.log(res.status); });
willWashTheDishes.catch((res) => { console.log(res.status); });
```

If the promise `willWashTheDishes` is resolved, then the output will be:

```
The dishes were washed!
```

On the other hand if it's rejected, then the output will be:

```
The dishes were all broken!
```

In practice, say when making API requests, the status code will typically be `404`, or `200`. And the data will be the data requested from the server.  

#### Refactoring Our Code

At this point, if we refactor the `increment` function using promises, we'll see that we haven't made much progress. 

```javascript
let result = 0; // Global variable that stores the result

function increment(num) {
  
  // Returns a Promise object
  return new Promise((resolve, reject) => { 
    if (result + num <= 20) {
      result += num;
      resolve();
    } else {
      reject();
    }
  });
  
}
```

The function `increment` now returns a `Promise` object which resolves if the `result` hasn't surpassed the value of `20`, in which case we call the equivalent of the `onSuccess` callback. Otherwise, the promise is rejected, in which case we call the equivalent of the `onFailure` callback. 

This is how we would now do the same task as before — that of incrementing `result` to `15` by `5`s.

```javascript
increment(5)
	// On success... (1)
  .then(() => {
  	increment(5)
  		// On success... (2)
  		.then(() => {
      	increment(5)
    	})
  		// On failure... (2)
  		.catch(() => {
      	console.log("I will not increment the value anymore!");
    	})
	})
	// On failure... (2)
	.catch(() => {
  	console.log("I will not increment the value anymore!");
	})

setTimeout(() => { console.log(`Result is: ${result}`);}, 1000)
```

```
Result is 15
```

> **Note:** The reason we delay the printing of `result` at the end by 1 second is actually because of the asynchronicity introduced by the promises. After executing `line 1`,  JavaScript asynchronously executes `line 15` without waiting for the resolution/rejection of the promises returned on lines `1`, `3`, and `5`. So, even though resolution/rejection are immediate in this example, they're still not fast enough to execute in time before control reaches `line 15`.

As we can see, we're still very much stuck in *the callback hell*. What helps us break out of it is *chaining promises*...



#### Chaining Promises

The nestedness arises from the fact that each callback in `then()` depends on the successful completion of the one before it, so it must be called within the scope of its predecessor. This is a pattern that arises naturally when we're making API requests. For instance, we could be making an initial request to an endpoint `/` grabbing a dataset of users and their user IDs. Then, we may want to make another call to a different endpoint `/UserID` in order to grab a single user's data using his/her ID. The second API request is contingent on the successful completion of the first... 

This is where the `Promise` object fully comes into play. What if the callback function of  `then()`  were to return a `Promise` object signifying the resolution or rejection of *that particular step* in the process? Then we could chain the consumers instead of nesting them!

Here's what that would look like:

```javascript
increment(5)
  .then(() => {
  	return increment(5) // Increment returns a promise
	})
	.then(() => {
  	return increment(5) // Increment returns a promise
	})
	.catch(() => {
  	console.log("I will not increment the value anymore!");
	})

setTimeout(() => { console.log(`Result is: ${result}`);}, 1000)
```

`Line 6` runs only if the promise on `line 3` is resolved, which runs only if the promise on `line 1` is resolved. So the conditional execution of the callbacks is preserved. `Line 9` catches any rejection that might occur on lines `1`, `3,` or `6`. So there's no longer a need for nested `catch()` consumers. 

Furthermore, since we're simply doing a `return` in each callback function, we can use the *implicit return* syntax to further refactor our code.

```javascript
increment(5)
  .then(() => increment(5))
	.then(() => increment(5))
	.catch(() => {
  	console.log("I will not increment the value anymore!");
	})

setTimeout(() => { console.log(`Result is: ${result}`);}, 1000)
```

Which is a significant improvement over *the callback hell* in terms of both length and readability!

### AJAX - Asynchronous JavaScript and XML

Post 2/2 from transfer on Nov 3, 2025. 


Back in the day, before single-page websites were a thing, it was only possible to fetch data from a server during a page reload. On reload, the client requested the relevant HTML, CSS, and JavaScript for each page from the server. Only upon completion of this request would the browser construct the page and run its associated JavaScript. 

Contrast this with single-page websites where we can make HTTP requests during the execution of the rest of the JavaScript, without needing to wait for their completion, selectively updating parts of the page while the rest of the page behaves and renders as expected. But how can JavaScript, a single-threaded language, make these time-consuming requests without pausing overall execution? 

As we've seen in a previous post, the browser's *Web APIs* help JavaScript run all sorts of asynchronous tasks. Things like executing callback functions, setting timers, and making API requests asynchronously. *AJAX* is simply that — a way for a client to make requests and continue the execution of JavaScript without having to halt and wait for a response. The *XML* in AJAX is a relic from the past referring to the old format used by the servers to send or receive data. Today *JSON* (*JavaScript Object Notation*) is the standard, prompting the question, "why don't we use the acronym *AJAJ* instead?"



### Working with JSON

As its name implies, JSON is a string format that's really close to JavaScript's own object notation. The key difference is that in JSON, the object keys are strings that must be contained within double-quotes. In fact, no single quotes are allowed anywhere in a valid JSON string. As far as values, all data types are allowed except for functional values.

Here's an example of a valid JSON string: 

```
{
	"user" : "Lambert",
  "id" : 13451345002,
  "pronouns" : ["He", "Him", "His"]
  "active" : true
}
```

When we make an API call, the server sends back JSON (i.e. a string that *almost* looks like a JavaScript object) which must be converted into a JavaScript object in order to be useful. Here's how we do that with a built-in method:

```javascript
const myData = JSON.parse(myJSON)
```

We can also do the reverse procedure to convert a JavaScript object into JSON if we're designing our own APIs that need to JSON over to our clients.  Here's how we can do that:

```javascript
const myJSON = JSON.stringify(myData)
```


### HTTP Requests Using Fetch

The *Fetch API* is a relatively new way to make HTTP requests (succeeding the *XMLHttpRequest API*). The main advantage of `fetch` is that it returns a `Promise` object, making for flatter requests!

Using the promise consumers `then()` and `catch()`, we can easily handle the data fetched by the request, or the error should the request fail. 

Let's see how we can communicate with the Star Wars API over at https://swapi.dev.

```javascript
fetch("https://swapi.dev/api/planets/")
```

Returns a promise:

...




# Debounce Mechanism

Context: This is unrelated to the previous two posts, but it's a neat trick that's got to live here for the time being...

A wrapper over `setTimeout(func, wait)` which calls a function `func` after a certain `wait` period. The advantage provided by the Debounce layer over direct use of `setTimeout(func, wait)` inside `addEventListener` is this:

Each scroll event doesn't trigger a new timeout without clearing the previous one first using `clearTimeout(timeout)`. It's irrelevant that `debounce` returns a function, it just does so to satisfy the higher-order requirement of the function `addEventListener`. Debounce also receives an annonymous function to pipe it to the `setTimeout` for use.

```ts
/**
 * Creates a debounced version of a function that delays execution
 * until after the specified wait time has elapsed since the last call.
 * 
 * @param func The function to debounce
 * @param wait The time to wait in milliseconds
 * @returns A debounced version of the original function
 */

function debounce<T extends (...args: any[]) => any> {
    let timeout;
    return function() {
        clearTimeout(timeout);
        timeout = setTimeout(func, wait);
    };
}

```